# Implementation of NN based Ranking Model on Slack Data

```/* @Author: Jagan Kaartik */```

In [1]:
import os
import re
import nltk

In [ ]:
os.getcwd()

In [ ]:
path = "../Data/Slack_Data/Dataset"

In [ ]:
filelist = os.listdir(path)
filelist

In [ ]:
filelist.pop(0)

In [ ]:
newpath = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data/Dataset"
os.chdir(newpath)

In [ ]:
os.getcwd()

In [ ]:
corpus = []
for i in filelist:
     with open("{}".format(i)) as f_input:
            corpus.append(f_input.read())

In [142]:
len(corpus)

6378

In [ ]:
corpus[0]

## Pre-Processing Corpus Data 

In [ ]:
newpath = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data"
os.chdir(newpath)

In [ ]:
# Remove \n in Corpus Txt

for c in range(0,len(corpus)):
    corpus[c] = re.sub('\n','',corpus[c])

In [ ]:
corpus[0]

## Lemmatization and Removal of Stop Words from Corpus

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 

for c in range(0,len(corpus)):
    word_list = nltk.word_tokenize(corpus[c])
    filtered_sentence = [w for w in word_list if not w in stop_words]
    corpus[c] = ' '.join([lemmatizer.lemmatize(w) for w in filtered_sentence])

In [ ]:
corpus[0]

### Writing Lemmatized Corpus to File for Efficient Retreival 

In [309]:
path = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data/Lemmatized Corpus/"
os.chdir(path)

In [310]:
for c in range(0,len(corpus)):
    with open('doc{}.txt'.format(c+1), 'w') as f:
        f.write("%s\n" % corpus[c])

### Inverted - Index

In [312]:
from collections import defaultdict

In [313]:
class InvertedIndex: 
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.index = defaultdict(list)
        self.documents = {}
        self.unique_id = 0
 
    def termLookup(self, word):
        word = word.lower()
        return [self.documents.get(id, None) for id in self.index.get(word)]
    
    def retIndex(self):           
        return self.index.items()
 
    def addTerm(self, document):
        for token in [t.lower() for t in nltk.word_tokenize(document)]:
            if self.unique_id not in self.index[token]:
                self.index[token].append(self.unique_id)
 
        self.documents[self.unique_id] = document
        self.unique_id += 1           

invObj = InvertedIndex(nltk.word_tokenize)

### Adding Corpus Data to Inverted Index

In [314]:
for c in range(0,100):
    invObj.addTerm(corpus[c])

In [315]:
res = invObj.termLookup("cancer")
print(len(res))

9


### View Inverted Index

In [316]:
invIndex = invObj.retIndex()

for k,v in invIndex:
    print("{} ---> {}".format(k,v)) 

link ---> [0, 1, 13, 16, 25, 29, 34, 37, 44, 48, 52, 53, 68, 71, 76, 79, 82, 83, 87, 90, 91, 94, 95]
national ---> [0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 45, 46, 47, 51, 52, 53, 54, 55, 56, 58, 59, 63, 66, 69, 70, 71, 72, 74, 76, 77, 79, 81, 82, 83, 87, 88, 89, 90, 91, 93, 94, 95]
cancer ---> [0, 7, 14, 15, 23, 29, 47, 79, 80]
institute ---> [0, 2, 5, 7, 9, 14, 15, 16, 19, 23, 25, 29, 30, 37, 38, 47, 48, 49, 56, 67, 70, 72, 74, 76, 77, 79, 82, 83, 88, 91, 98]
center ---> [0, 2, 3, 7, 8, 10, 11, 14, 15, 16, 18, 22, 23, 25, 27, 29, 30, 33, 34, 36, 37, 38, 39, 40, 42, 44, 45, 46, 47, 48, 52, 53, 54, 55, 56, 58, 60, 61, 62, 63, 64, 67, 69, 70, 71, 72, 73, 74, 75, 76, 79, 81, 82, 89, 90, 91, 99]
research ---> [0, 2, 6, 7, 9, 10, 11, 14, 15, 16, 18, 23, 24, 25, 27, 30, 34, 37, 38, 39, 40, 42, 44, 47, 48, 52, 55, 56, 58, 59, 62, 68, 69, 70, 71, 72, 74, 75, 76, 79, 80, 82, 83, 87, 88, 89, 90, 91, 98]
contact 

tank.an ---> [2]
advantage ---> [2, 14, 16, 20, 23, 24, 37, 44, 52, 53, 87]
last ---> [2, 3, 4, 10, 11, 13, 15, 17, 18, 19, 20, 21, 22, 23, 25, 27, 30, 32, 33, 34, 36, 39, 42, 44, 47, 49, 51, 52, 53, 54, 55, 56, 63, 66, 78, 79, 82, 83, 85, 87, 90, 92, 94, 98, 99]
longer ---> [2, 7, 9, 16, 24, 29, 33, 37, 39, 42, 44, 47, 53, 55, 56, 58, 63, 64, 69, 75, 79, 82, 87]
engines.^ ---> [2]
after ---> [2, 25, 37, 53, 54, 56, 83, 87]
conducting ---> [2, 3, 7, 23, 37, 40, 52, 55, 69, 70, 71, 73, 87]
dekalb ---> [2]
georgia ---> [2, 9, 16, 43, 44, 76]
noted ---> [2, 7, 8, 25, 30, 32, 34, 37, 42, 48, 54, 56, 58, 62, 75, 79, 82, 87]
acquisition ---> [2, 16, 39, 88]
steep ---> [2, 16]
long-term ---> [2, 7, 15, 16, 17, 23, 39, 45, 53, 55, 58, 60, 64, 69, 72, 73, 74, 75, 79, 87]
saving ---> [2, 16, 40, 47, 53, 90]
outweighed ---> [2, 37, 79]
short-term ---> [2, 16, 23, 39, 48, 53, 58, 60, 70, 75, 79, 82, 91]
costs.^ ---> [2]
6.estimated ---> [2]
din ---> [2]
266,000 ---> [2]
among ---> [2, 5, 7, 8, 10,

use^a ---> [2]
autos ---> [2]
28.7 ---> [2]
33.1 ---> [2]
67.2 ---> [2]
21.9 ---> [2]
31.9 ---> [2]
4.5 ---> [2]
1.2 ---> [2, 6, 16, 69]
62.7 ---> [2]
32.5 ---> [2]
31.3 ---> [2]
29.9 ---> [2]
4.2 ---> [2, 69]
4.7 ---> [2, 48]
0.4 ---> [2]
19.2 ---> [2]
26.4 ---> [2]
29.5 ---> [2]
9.1 ---> [2]
0.1 ---> [2]
24.4 ---> [2]
19.3 ---> [2]
1.4 ---> [2, 7, 69]
4.6 ---> [2, 63]
2.7 ---> [2]
0.2 ---> [2]
19.8 ---> [2]
16.6 ---> [2, 7]
7.1 ---> [2]
0.6 ---> [2]
7.2 ---> [2]
8.0 ---> [2]
2.9 ---> [2]
1.3 ---> [2, 39, 53, 64, 69]
6.7 ---> [2]
0.5 ---> [2]
0.8 ---> [2, 69]
100.0 ---> [2]
93,310 ---> [2]
18,941 ---> [2]
^areported ---> [2]
survey.these ---> [2]
represent ---> [2, 5, 8, 15, 16, 23, 29, 37, 39, 40, 55, 58, 64, 75, 80, 81, 82, 83, 87, 89, 93, 96]
entire ---> [2, 5, 9, 19, 23, 39, 42, 53, 54, 55, 56, 58, 59, 63, 69, 83, 87, 89, 98, 99]
population ---> [2, 15, 16, 18, 19, 23, 25, 29, 30, 33, 39, 42, 47, 48, 54, 55, 56, 58, 64, 69, 71, 75, 79, 82, 87, 98, 99]
shown ---> [2, 11, 16, 19, 23

probation.new ---> [3]
birth ---> [3, 13, 32, 37, 47, 79, 87, 98]
missionary ---> [3, 33, 43]
charles ---> [3, 10, 16, 19, 37, 76, 80, 87]
fire.boxes ---> [3]
sitting ---> [3, 8, 29, 56, 59, 79]
wall ---> [3, 8, 10, 28, 29, 48, 51, 60, 79, 87]
spread ---> [3, 8, 9, 12, 16, 25, 34, 39, 52, 54, 64, 69, 83]
minimal ---> [3, 16, 48, 53, 66, 99]
church.two ---> [3]
charges.each ---> [3]
probation.57.our ---> [3]
lady ---> [3, 17, 30, 85]
queen ---> [3, 16]
heaven ---> [3, 8]
damaged.the ---> [3]
our ---> [3, 16, 19, 39, 44, 47, 49, 56, 58, 99]
proximity ---> [3, 16, 40, 87]
school.the ---> [3]
60,000.two ---> [3]
18.58.cypress ---> [3]
zachary ---> [3]
59.saint ---> [3]
paul ---> [3, 16, 25, 26, 37, 56, 58, 76, 87]
60.sweet ---> [3]
61.saint ---> [3]
benevolent ---> [3]
society ---> [3, 8, 9, 14, 15, 23, 33, 37, 47, 48, 54, 55, 56, 72, 76, 79, 82, 83, 86, 88, 91]
arsonist ---> [3, 33, 93]
brought ---> [3, 8, 25, 29, 33, 37, 44, 53, 54, 55, 56, 57, 62, 66, 69, 87, 89]
churches.in ---> [3]
st

retire ---> [4, 53, 89]
hold ---> [4, 7, 8, 15, 16, 17, 19, 23, 26, 32, 33, 37, 40, 52, 53, 54, 55, 60, 62, 63, 65, 78, 87, 92]
accountable ---> [4, 39, 89, 99]
profiteering ---> [4, 40]
ease ---> [4, 30, 39, 63, 69, 79]
burden ---> [4, 7, 8, 16, 37, 39, 48, 53, 62, 89]
bear ---> [4, 7, 37, 40, 56, 58, 76, 79, 87]
lion ---> [4, 76, 87]
harm ---> [4, 8, 24, 33, 37, 39, 47, 48, 52, 58, 69, 87]
inflicted ---> [4, 37, 55]
gouger ---> [4]
pillaged ---> [4]
yet.we ---> [4]
dozen ---> [4, 17, 30, 42, 44, 53, 54, 74, 87]
active ---> [4, 7, 8, 14, 16, 19, 23, 30, 33, 37, 39, 40, 48, 53, 55, 56, 58, 60, 63, 69, 75, 79, 89, 97]
lawsuit ---> [4, 33, 37, 55]
back ---> [4, 8, 9, 15, 18, 19, 25, 30, 32, 33, 37, 39, 40, 42, 43, 44, 49, 50, 52, 53, 54, 56, 59, 63, 65, 73, 74, 75, 76, 79, 80, 87, 96, 99]
picked ---> [4, 13, 16, 25, 87]
pocket ---> [4, 23, 28, 39]
forthcoming ---> [4, 87]
information.he ---> [4]
cooperating ---> [4, 14, 30, 43]
manipulation.under ---> [4]
allowed ---> [4, 8, 16, 18, 19, 

...

dial ---> [73]
operators.once ---> [73]
perused ---> [73]
leaders.furthermore ---> [73]
on.the ---> [73]
hotline.this ---> [73]
24-hours ---> [73]
personnel.all ---> [73]
wallet ---> [73]
emergency.depending ---> [73]
designees ---> [73]
gained.without ---> [73]
disposal.callers ---> [73]
uninformed ---> [73]
crisis.finally ---> [73]
tying ---> [73]
scenarios.the ---> [73]
groundwork ---> [73, 99]
table-top ---> [73]
simulation.some ---> [73]
in-country ---> [73]
partners.too ---> [73]
advertising.due ---> [73]
diligence ---> [73]
unearth ---> [73]
hassle ---> [73]
lives.contracting ---> [73]
medevac ---> [73]
organization.in-country ---> [73]
dicey ---> [73]
services.considering ---> [73]
kidnap ---> [73]
ransom ---> [73, 76]
insurance.the ---> [73]
painless ---> [73]
speedy ---> [73]
confusing ---> [73]
time.kidnappings ---> [73]
amounts.all ---> [73]
readiness ---> [73]
eeurope.gif ---> [74]
11.5 ---> [74]
boiling ---> [74]
siberian ---> [74]
bilibino.the ---> [74]
legacythe ---> [7

5916 ---> [76]
64111muse ---> [76]
musicians ---> [76]
cresent ---> [76]
14214monadnock ---> [76]
munsonville ---> [76]
03457morning ---> [76]
3672 ---> [76]
19807motif ---> [76]
43210muddy ---> [76]
1244 ---> [76]
65102the ---> [76]
museletter ---> [76]
lanes ---> [76]
wrentham ---> [76]
02093-0323music ---> [76]
semfolk ---> [76]
arnold ---> [76]
bedford ---> [76]
02740musical ---> [76]
ashbrook ---> [76]
8310 ---> [76]
20907muskeg ---> [76]
778 ---> [76]
03755neffa ---> [76]
02140nhac ---> [76]
viet ---> [76]
vietnamese ---> [76]
44240name-ye-shayda ---> [76]
shayda ---> [76]
19336 ---> [76]
gaithersburg ---> [76, 79]
20879nashville ---> [76]
4781 ---> [76]
37216natchitoches ---> [76]
71457national ---> [76]
d.c.blues ---> [76]
77315 ---> [76]
d.c.20013-7715national ---> [76]
fiddlernational ---> [76]
54055 ---> [76]
39208nevada ---> [76]
germain ---> [76]
7969 ---> [76]
rodeo ---> [76]
89119new ---> [76]
02141new ---> [76]
brattleboro ---> [76]
05301the ---> [76]
jubilee ---> [76]


prostateif ---> [80]
holistic ---> [80]
remedy.for ---> [80]
hyperlink ---> [80]
testimonials ---> [80]
usersi ---> [80]
fatigue.the ---> [80]
wonderful.the ---> [80]
immediate.. ---> [80]
brother-in ---> [80]
essiac.i ---> [80]
gone.. ---> [80]
.my ---> [80]
nephew ---> [80, 89]
cancer.. ---> [80]
remission ---> [80]
cancer.under ---> [80]
chemotherapy.i ---> [80]
psa ---> [80]
widespread.. ---> [80]
it.well ---> [80]
remarkable.i ---> [80]
sixty-two ---> [80]
sixteen ---> [80]
back.. ---> [80]
cancer.i ---> [80]
session.every ---> [80]
so-o-o ---> [80]
hemo ---> [80]
fantastic ---> [80]
patient.. ---> [80]
canada.after ---> [80]
'essiac ---> [80]
cancer'.in ---> [80]
1950s ---> [80, 89]
brusch ---> [80]
massachusetts.after ---> [80]
.will ---> [80]
xxxxxxx ---> [80]
illness.for ---> [80]
gary ---> [80]
glum ---> [80]
die.they ---> [80]
carini ---> [80]
histoplasmosis.their ---> [80]
ten.the ---> [80]
with.he ---> [80]
ounce ---> [80]
day.by ---> [80]
died.dr ---> [80]
alive.they --->

education.some ---> [87]
shifting ---> [87]
help.ordinary ---> [87]
research.dfg ---> [87]
fund.non-hunting ---> [87]
enthusiast ---> [87]
wildlands ---> [87]
3211 ---> [87]
95816 ---> [87]
checkoff ---> [87]
fund.see ---> [87]
540.-second ---> [87]
pioneering ---> [87]
country.situations ---> [87]
ourselves.and ---> [87]
out.keep ---> [87]
night.put ---> [87]
top.light ---> [87]
property.use ---> [87]
them.-finally ---> [87]
safe.keep ---> [87]
areas.all ---> [87]
children.clark ---> [87]
prey.stand ---> [87]
tall.carry ---> [87]
stick.if ---> [87]
run.don ---> [87]
crouch.never ---> [87]
children.appear ---> [87]
can.throw ---> [87]
stones.shout ---> [87]
advise.fjelline ---> [87]
posture ---> [87]
mistook ---> [87]
minute.that ---> [87]
sites.her ---> [87]
sens ---> [87]
smell ---> [87]
see.she ---> [87]
humans.but ---> [87]
size.men ---> [87]
bigger.or ---> [87]
humans.you ---> [87]
yourself.no ---> [87]
blame ---> [87]
it.haven ---> [87]
free-lance ---> [87]
cool.dfg ---> [87]
com

33131 ---> [92]
1715 ---> [92]
21203.as ---> [92]
9:00 ---> [92, 94]
salon ---> [92]
marriott ---> [92]
airport.interested ---> [92]
hearing.persons ---> [92]
202/377-2862 ---> [92]
4.instead ---> [92]
1986.a ---> [92]
1529 ---> [92]
20230.dated ---> [92]
1986.john ---> [92]
doc.86-3291 ---> [92]
2-13-86 ---> [92]
8:45 ---> [92]
3510-ds-m ---> [92]
treasemblem.jpg ---> [93]
622-2960 ---> [93]
514-2007 ---> [93]
d.c.c ---> [93]
decline.task ---> [93]
vigilance ---> [93]
well-publicized ---> [93]
efforts.in ---> [93]
worship.in ---> [93]
140.these ---> [93]
1999.this ---> [93]
2000.as ---> [93]
36.2 ---> [93]
945 ---> [93]
investigated.the ---> [93]
horrific ---> [93]
fires ---> [93]
sear ---> [93]
law.for ---> [93]
431 ---> [93]
1999.sentencing ---> [93]
prison.this ---> [93]
created.ballinger ---> [93]
georgia.the ---> [93]
www.atf.treas.gov/pub ---> [93]
00-542 ---> [93]
train/subway ---> [94]
fare ---> [94]
4.00.use ---> [94]
stations.you ---> [94]
information.hilton ---> [94]
tysons

In [317]:
len(invIndex)

42609

### Lookup for Terms in Inverted Index

In [ ]:
term = "cancer"
lookupResult = invObj.termLookup(term)

In [ ]:
print("No of Documents with Term {} is {} ".format(term,len(lookupResult)))

### Saving Invertex Index to CSV

In [ ]:
import csv
w = csv.writer(open("InvertedIndex.csv", "w"))
for key, val in invIndex:
    w.writerow([key, val])

### Size of Invertex Index with 100 corpus documents.

In [ ]:
print(len(invIndex))

### Load Lemmatized Corpus

In [2]:
path = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data/Lemmatized Corpus/"
os.chdir(path)

In [3]:
filelist = os.listdir(path)
filelist

['doc1.txt',
 'doc10.txt',
 'doc100.txt',
 'doc1000.txt',
 'doc1001.txt',
 'doc1002.txt',
 'doc1003.txt',
 'doc1004.txt',
 'doc1005.txt',
 'doc1006.txt',
 'doc1007.txt',
 'doc1008.txt',
 'doc1009.txt',
 'doc101.txt',
 'doc1010.txt',
 'doc1011.txt',
 'doc1012.txt',
 'doc1013.txt',
 'doc1014.txt',
 'doc1015.txt',
 'doc1016.txt',
 'doc1017.txt',
 'doc1018.txt',
 'doc1019.txt',
 'doc102.txt',
 'doc1020.txt',
 'doc1021.txt',
 'doc1022.txt',
 'doc1023.txt',
 'doc1024.txt',
 'doc1025.txt',
 'doc1026.txt',
 'doc1027.txt',
 'doc1028.txt',
 'doc1029.txt',
 'doc103.txt',
 'doc1030.txt',
 'doc1031.txt',
 'doc1032.txt',
 'doc1033.txt',
 'doc1034.txt',
 'doc1035.txt',
 'doc1036.txt',
 'doc1037.txt',
 'doc1038.txt',
 'doc1039.txt',
 'doc104.txt',
 'doc1040.txt',
 'doc1041.txt',
 'doc1042.txt',
 'doc1043.txt',
 'doc1044.txt',
 'doc1045.txt',
 'doc1046.txt',
 'doc1047.txt',
 'doc1048.txt',
 'doc1049.txt',
 'doc105.txt',
 'doc1050.txt',
 'doc1051.txt',
 'doc1052.txt',
 'doc1053.txt',
 'doc1054.txt',
 'd

In [4]:
corpus = []
for i in filelist:
     with open("{}".format(i)) as f_input:
            corpus.append(f_input.read())

In [6]:
#Lemmatized Corpus
corpus[0]

"Link National Cancer Institute Center Cancer Research Link Contact CCR Link CCR Homepage Link NCI home Link NIH home _______________ Search Comparative Oncology Program Home Introduction Specific Aims Time Line- For Implementation Background The CCR - Comparative Oncology Program compliment number new CCR initiative designed improve translational process.Program Announcement - 9/2003 Comparative Oncology refers study naturally developing cancer animal model human disease.A significant under-utilized group naturally occurring cancer develops pet animal , primarily cat dogs.These large animal cancer share many feature human cancer including tumor histology , genetics , response conventional therapy biological behavior.Examples model include : Osteosarcoma Non-Hodgkin 's Lymphoma Breast Cancer Head Neck Carcinoma Prostate Cancer Soft Tissue Sarcoma Melanoma Virally-Induced-LymphomaThrough design clinical trial include pet animal broader understanding treatment biology cancer attained.Man

### Convert Corpus to Vectors

In [7]:
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize 

In [26]:
def processCorpus(train):
    traincorp = []
    for i in train:
        traincorp.append(word_tokenize(i))
    return traincorp

In [35]:
train = corpus

In [36]:
traincorp = processCorpus(train)

In [37]:
modelW2V = Word2Vec(traincorp, min_count=1)

In [38]:
modelW2V.save('modelfull.bin')

In [39]:
path = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data/"

* Refer : https://machinelearningmastery.com/develop-word-embeddings-python-gensim/ 
* Refer : https://www.geeksforgeeks.org/tokenize-text-using-nltk-python/
* Refer : https://stackoverflow.com/questions/55713132/how-to-tokenize-a-list-of-lists-in-python

### Model Based Similarity

##### Score of using our Model to compare Cancer with similar words such as Ovarian, Tumor

In [53]:
modelW2V.wv.similarity('cancer', 'tumor') 

0.8035345

In [52]:
modelW2V.wv.similarity('cancer','ovarian')

0.86470807

##### Score of using our Model to compare Cancer with dissimilar words such as Cloud

In [51]:
modelW2V.wv.similarity('cancer','cloud')

0.20746103

##### Finding top 10 most similar words to word Cancer in our dataset

In [49]:
v1 = "cancer"
modelW2V.wv.most_similar(positive=v1)

[('prostate', 0.9280345439910889),
 ('ovarian', 0.8647080659866333),
 ('cancer.The', 0.8155494928359985),
 ('tumor', 0.8035344481468201),
 ('breast', 0.792565107345581),
 ('cancers.The', 0.781282365322113),
 ('colorectal', 0.77799391746521),
 ('cancer.In', 0.7480220794677734),
 ('malignant', 0.7331823706626892),
 ('cancer.Dietary', 0.7293593883514404)]

## Doc2Vec Model for Finding out document similarity

In [64]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [65]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(corpus)]

### Traning the Model

In [1]:
max_epochs = 10
vec_size = 20 
alpha = 0.025

In [228]:
model = Doc2Vec(size=vec_size,alpha=alpha, min_alpha=0.00025,min_count=50,dm =1)

/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [229]:
model.build_vocab(tagged_data)

In [230]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0


/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [232]:
model.save("min50.model")

### Cosinie Similarity between Documents

In [233]:
from scipy import spatial

In [234]:
vec1 = model.infer_vector(corpus[1].split())
vec2 = model.infer_vector(corpus[3].split())

In [235]:
similairty = spatial.distance.cosine(vec1, vec2)
similairty

0.6142655909061432

In [239]:
new_sentence = "i love dogs".split(" ") 

In [240]:
model.docvecs.most_similar(positive=[model.infer_vector(new_sentence)],topn=5)

[('5235', 0.7422172427177429),
 ('4870', 0.7328481674194336),
 ('95', 0.7185875773429871),
 ('5868', 0.7118589878082275),
 ('1954', 0.6987151503562927)]

In [244]:
corpus[5868]

"DL-1 Rev . 12/95DOG IDENTIFICATION License No.Date Issues Dog Breed Dog Color ( ) Other ID Markings Dog 's Yr. Birth Last 2 Digits Dog 's Name Chk Code Expiration Date Code Code ( ) New York State Department Agriculture Markets Division Animal Industry 1 Winners Circle - Albany , New York 12235 518-457-2728DOG LICENSE Issuing County Code/TCV CodeRABIES CERTIFICATE REQUIRED Rabies Vaccine : Manufacturer __________________________ Serial Number __________________________LICENSE TYPEOne Year Vacc.Three Year Vacc.ORIGINAL RENEWAL TRANSFER OF OWNERSHIPDate Vaccinated ______________________ Veterinarian ______________________________Owner Identification ( Person harbor keep dog ) : Last First Middle InitialOWNER 'S PHONE NO.Area CodeMailing Address : House No.Street R.D . No . P.O . Box No.Phone No.CityStateZipCounty CodeCountyTown , City VillageTown , City , Vil . CodeTYPE OF LICENSE 1.2.3.Female , spayed Male , unneutered 6 month 4.6 mo . 6 month Male , neuteredState fee Spay/Neuter Fee 2

In [245]:
model = Doc2Vec.load("d2v.model")

In [246]:
query = "i love dogs".split(" ") 

In [250]:
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

[('638', 0.7319365739822388),
 ('1041', 0.6784754991531372),
 ('2317', 0.6569496989250183),
 ('5235', 0.6252104043960571),
 ('2664', 0.6232786178588867)]

In [251]:
#Corpus Regarding Euthanesia, Animal Health, Veterinarian etc. These WORDS are Similar in semantic similarity with Dogs!
corpus[638]

"ISSN : 1052-5378 AWIC Animal Euthanasia Provided Animal Welfare Information Center United States Department Agriculture National Agricultural Library United States Department Agriculture National Agricultural Library 10301 Baltimore Blvd.Beltsville , Maryland 20705-2351SRB 93-06Special Reference Briefs SeriesAnimal EuthanasiaUpdated Special Reference Briefs Series no.SRB 98-01Michael D. Kreger Cynthia Petrie Smith Jennifer Lyons-CarterAnimal Welfare Information CenterJuly 1993National Agricultural Library Cataloging Record : Kreger , Michael D. Animal euthanasia . ( Special reference brief ; 93-06 ) 1.Euthanasia animal -- Bibliography.I . Smith , Cynthia Petrie.II.Lyons-Carter , Jennifer.III.Title.aS21.D27S64 no.93-06ANIMAL EUTHANASIATABLE OF CONTENTS* Introduction.. . . . . . . . . ... . . . . . . . . . . . .ii * General Veterinary Use . . . . . . . . . . . . . . . . . . ..1 * Teaching.. . . . . . . . . . . . . . . . . . . . . . . . ..7 * Human/Animal Bond.. . . . . . . . . . . . . .

#### Benefits of Semantic Similarity based Ranking is, documents related in semantic similarity with the query is returned. Using this to initialty filter results (documents) would be ideal.

In [261]:
query = "Healthcare Research".split(" ") 

In [263]:
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

[('5508', 0.6469775438308716),
 ('5800', 0.6222918033599854),
 ('5692', 0.6181532144546509),
 ('6304', 0.6029208898544312),
 ('997', 0.5998862981796265)]

In [265]:
corpus[5508]

"Annual Report COMMONWEALTH OF VIRGINIA 2002 Table Contents Executive Summary ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... .ii State-wide Reach ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... .iii Future Direction ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... .. iv Activities Governor 's Office Substance Abuse Prevention ... ... ... ... ... 1 Statutory Authority ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... 1 GOSAP Organization Operation ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... 1 Funding ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... 2 GOSAP Philo

In [294]:
query = "safety".split(" ") 
model.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

[('107', 0.7906770706176758),
 ('2530', 0.7669410109519958),
 ('3664', 0.7627139687538147),
 ('2932', 0.7612925171852112),
 ('4881', 0.7560254335403442)]

In [295]:
corpus[107]

"Skip common site navigation header United States Environmental Protection Agency Clean Air Markets - Environmental Issues Recent Additions | Contact Us | Print Version Search : __________ Run Search Begin Hierarchical Links EPA Home The reported damage typically occurs horizontal surface appears irregularly shaped , permanently etched areas.The damage best detected fluorescent lamp , easily observed dark colored vehicle , appears occur evaporation moisture droplet.In addition , evidence suggests damage occurs frequently freshly painted vehicles.Usually damage permanent ; occurred , solution repaint.The general consensus within auto industry damage caused form environmental fallout . `` Environmental fallout , '' term widely used auto coating industry , refers damage caused air pollution ( e.g. , acid rain ) , decaying insect , bird droppings , pollen , tree sap.The result laboratory experiment least one field study demonstrated acid rain scar automotive coatings.Furthermore , chemical

In [301]:
newmodel = Doc2Vec.load("d2vmin50.model")

[('4981', 0.6771769523620605),
 ('1759', 0.6741712093353271),
 ('4465', 0.6733023524284363),
 ('5965', 0.6625094413757324),
 ('3294', 0.6572967767715454)]

In [307]:
query = "dog".split(" ") 
newmodel.docvecs.most_similar(positive=[model.infer_vector(query)],topn=5)

[('748', 0.7187636494636536),
 ('1324', 0.7062160968780518),
 ('535', 0.7006916403770447),
 ('3253', 0.6962764859199524),
 ('544', 0.6899124383926392)]

In [308]:
corpus[748]

"810 KAR 1:001.Definitions.RELATES TO : KRS 230.210 230.360STATUTORY AUTHORITY : KRS 230.260NECESSITY , FUNCTION , AND CONFORMITY : To regulate condition thoroughbred racing shall conducted Kentucky.The function administrative regulation define term used commission 's administrative regulations.Section 1.Definitions . ( 1 ) `` Added money '' mean cash , exclusive trophy award , added association stake fee paid subscriber form total purse stake race . ( 2 ) `` Age '' mean number year since horse foaled , reckoned horse foaled January 1 year horse foaled . ( 3 ) `` Arrears '' mean sum due licensee reflected account horseman 's bookkeeper , including subscription , jockey fee , forfeiture , default incident administrative regulation . ( 4 ) `` Association '' mean person legal entity , required licensed KRS 230.300 conduct race meeting , used herein , association conducting race meeting rule applicable . ( 5 ) `` Authorized agent '' mean person currently licensed agent licensed owner princ

In [2]:
import os

In [3]:
os.getcwd()

'/Volumes/JK/AI-Search-Engine/src/Models'

In [4]:
path = "/Volumes/JK/AI-Search-Engine/src/Data/Slack_Data/Models/"

In [5]:
os.chdir(path)

In [6]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [7]:
model = Doc2Vec.load("d2v.model")

In [8]:
model